# Basic Entropy Pooling
#### The idea is to create a simple, fully specified, multi-variate model for the core collateral portfolio. In it's most basic format, a MV-normal model shall suffice. This constitute the prior. 
#### The views on the simple model can be of the following form:
##### ->> Means drop to zero or are negative, for all assets 
##### ->> Variances increase, covariances increase with y%, for all asset pairs 
##### ->> Similar such BS, relating mean and variance and possibly other factors influencing the model (similar to the inflation example in the Butterfly trading task [Question:Is this even possible in the simple Meucci setup?])
#### Using the posterior distribution, solve the optimization problem to obtain possible stress-scenario implied collateral holdings

In [1]:
%matplotlib inline
# coding=utf-8

# Python 3 style division operator w/o the need to convert to float
from __future__ import division

import sys
sys.path.extend(['/Users/Dim/Desktop/school_folder/masters_thesis/gitCodeRepo/codePython/collateralOptimizer/'])
import pandas as pd
import numpy as np
from sklearn.covariance import *

In [85]:
# Function prepareProblem: this function constructs the mean vector & variance-covariance matrix
# @param filePath:
# @param shrinkage: whether to apply the Ledoit, Wolf (2004) linear shrinkage estimator
# @param subsetSize: how many days of historical data to use for estimation purpose
def prepareProblem(filePath, shrinkage=False, subset=False, subsetSize=0):
    # Import data from .csv
    df = pd.read_csv(filePath, sep=';')
    df.index = df.date
    df = df.drop('date', axis=1)
    
    # Estimate mean
    mu = df.mean(axis=0)

    # Subset, if called via subset == True
    if subset == True:
        df = df.tail(subsetSize)

    # Estimate covariance using Empirical/MLE
    # Expected input is returns, hence set: assume_centered = True
    mleFitted = empirical_covariance(X=df, assume_centered=True)
    sigma = mleFitted

    if shrinkage == True:
        # Estimate covariance using LedoitWolf, first create instance of object
        lw = LedoitWolf(assume_centered=True)
        lwFitted = lw.fit(X=df).covariance_
        sigma = lwFitted

    return {'Mu_':mu, 'S_':sigma}

def getISIN(filePath):
    # Import data from .csv
    df = pd.read_csv(filePath, sep=';')
    df.index = df.date
    df = df.drop('date', axis=1)
    
    # Extract names
    return df.columns 

# Set path
filePath = '/Users/Dim/Desktop/school_folder/masters_thesis/gitCodeRepo/data/noStaleX_returnsData_20160825.csv'

In [3]:
# Collect data, prepare mean and covariance matrix
dataDict = prepareProblem(filePath=filePath, shrinkage=True, subset=True, subsetSize=250)

In [44]:
# Spec. a MV normal model using the estimated mean and covariances
N   = dataDict['Mu_'].shape[0]
J   = int(1e3)
p   = [1/J] * J
mu  = np.asanyarray(dataDict['Mu_']).reshape(1, dataDict['Mu_'].__len__())
var = np.random.multivariate_normal(mean=dataDict['Mu_'], 
                                    cov=dataDict['S_'], 
                                    size=int(J/2)) # Antithetic sampling to reduce variance

# Construct X = mu_i + var_i model 
X = np.dot(np.ones(shape=(J, 1)), mu) + np.concatenate([var, -var], axis=0)

# Spec. views on the MV model
